In [7]:
import os
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
# 1) Define tools (here: Tavily web search)
tools = [TavilySearchResults(max_results=1)]


# 2) Load the standard ReAct prompt from the LangChain Hub
prompt_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(prompt_template)

In [19]:
# 3) Choose an LLM

os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")

model_name = "llama3.2:3b"
max_tokens = 100
llm = ChatOllama(model=model_name, base_url=OLLAMA_URL, num_predict=max_tokens, temperature=0)

In [22]:
# 4) Create the ReAct agent

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# 5) Wrap it in an AgentExecutor
agent_executor  = AgentExecutor(agent=agent, tools=tools, verbose=False)

# 6) Run the agent
result = agent_executor.invoke({"input": "What is LangChain?"})
print(result["output"])

Agent stopped due to iteration limit or time limit.
